In [1]:
import altair
import matplotlib.pyplot as plt
from numba import njit, int64, float64
from numba.typed import List
from numba.types import Tuple, float64
import polars as pl
import numpy as np
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest, LIMIT, GTC, NONE, NEW, FILLED, CANCELED, EXPIRED, GTX, Recorder, ROIVectorMarketDepthBacktest, BUY_EVENT

In [10]:
# beerusdt = np.load('beerusdt_20240601.npz')['data']
binance = np.load('binance_20240601.npz')['data']
# beerusdt
np.set_printoptions(suppress=True)
binance

array([(3489660930, 1717200000131000000, 1717200000142177000, 67577.9,  0.004, 0, 0, 0.),
       (3489660930, 1717200000136000000, 1717200000147175000, 67577.9,  0.004, 0, 0, 0.),
       (3489660930, 1717200000140000000, 1717200000152690000, 67577.9,  0.002, 0, 0, 0.),
       ...,
       (3758096385, 1717286399962000000, 1717286399964864000, 67791.7,  0.136, 0, 0, 0.),
       (3758096385, 1717286399962000000, 1717286399964864000, 67792. ,  0.184, 0, 0, 0.),
       (3758096385, 1717286399962000000, 1717286399964864000, 67799.2, 14.053, 0, 0, 0.)],
      dtype=[('ev', '<u8'), ('exch_ts', '<i8'), ('local_ts', '<i8'), ('px', '<f8'), ('qty', '<f8'), ('order_id', '<u8'), ('ival', '<i8'), ('fval', '<f8')])

In [3]:
asset = (
    BacktestAsset()
        .data(binance)
        .linear_asset(1.0)
        .constant_latency(10_000_000, 10_000_000)
        .risk_adverse_queue_model()
        .no_partial_fill_exchange()
        .trading_value_fee_model(-0.0005, 0.0005)
        .tick_size(0.001)
        .lot_size(0.1)
        .last_trades_capacity(2)
        .roi_lb(67000)
        .roi_ub(68500)
)

In [4]:
@njit
def orderbookimbalance(hbt, out):
    roi_lb_tick = int(round(67000 / 0.001))
    roi_ub_tick = int(round(68500 / 0.001))

    while hbt.elapse(60 * 1e9) == 0:
        depth = hbt.depth(0)

        mid_price = (depth.best_bid + depth.best_ask) / 2.0

        sum_ask_qty_50bp = 0.0
        sum_ask_qty = 0.0
        for price_tick in range(depth.best_ask_tick, roi_ub_tick + 1):
            if price_tick < roi_lb_tick or price_tick > roi_ub_tick:
                continue
            t = price_tick - roi_lb_tick

            ask_price = price_tick * depth.tick_size
            depth_from_mid = (ask_price - mid_price) / mid_price
            if depth_from_mid > 0.01:
                break
            sum_ask_qty += depth.ask_depth[t]

            if depth_from_mid <= 0.005:
                sum_ask_qty_50bp = sum_ask_qty


        sum_bid_qty_50bp = 0.0
        sum_bid_qty = 0.0
        for price_tick in range(depth.best_bid_tick, roi_lb_tick - 1, -1):
            if price_tick < roi_lb_tick or price_tick > roi_ub_tick:
                continue
            t = price_tick - roi_lb_tick

            bid_price = price_tick * depth.tick_size
            depth_from_mid = (mid_price - bid_price) / mid_price
            if depth_from_mid > 0.01:
                break
            sum_bid_qty += depth.bid_depth[t]

            if depth_from_mid <= 0.005:
                sum_bid_qty_50bp = sum_bid_qty

        imbalance_50bp = sum_bid_qty_50bp - sum_ask_qty_50bp
        imbalance_1pct = sum_bid_qty - sum_ask_qty
        imbalance_tob = depth.bid_depth[depth.best_bid_tick - roi_lb_tick] - depth.ask_depth[depth.best_ask_tick - roi_lb_tick]

        # out.append((hbt.current_timestamp, depth.bid_depth[depth.best_ask_tick], depth.best_bid_tick, depth.bid_depth[depth.best_bid_tick - roi_lb_tick]))
        out.append((hbt.current_timestamp, imbalance_tob, imbalance_50bp, imbalance_1pct))

        # print(f'Mid Price: {str(mid_price)}, TOB Imbalance: {imbalance_tob}, 0.5% Imbalance: {imbalance_50bp}, 1% Imbalance: {imbalance_1pct}')
    return True

hbt = ROIVectorMarketDepthBacktest([asset])

tup_ty = Tuple((float64, float64, float64, float64))
out = List.empty_list(tup_ty)
orderbookimbalance(hbt, out)
_ = hbt.close()

df = pl.DataFrame(out).transpose()
df.columns = ['Local Timestamp', 'TOB Imbalance', '0.5% Imbalance', '1% Imbalance']
df = df.with_columns(
    pl.from_epoch('Local Timestamp', time_unit='ns')
)

df


Local Timestamp,TOB Imbalance,0.5% Imbalance,1% Imbalance
datetime[ns],f64,f64,f64
2024-06-01 00:01:00.131000064,-6.708,41.075,119.283
2024-06-01 00:02:00.131000064,-9.981,6.306,122.064
2024-06-01 00:03:00.131000064,-5.382,17.741,146.548
2024-06-01 00:04:00.131000064,7.068,24.097,180.839
2024-06-01 00:05:00.131000064,15.039,60.396,148.071
…,…,…,…
2024-06-01 23:55:00.131000064,20.716,-43.112,217.23
2024-06-01 23:56:00.131000064,-18.958,-110.777,151.001
2024-06-01 23:57:00.131000064,1.0,-75.499,187.77


In [5]:
import hvplot.polars
hvplot.extension('bokeh')
df.hvplot(x='Local Timestamp', width=1000, height=500)


:NdOverlay   [Variable]
   :Curve   [Local Timestamp]   (value)

In [6]:
@njit
def print_last_trades(hbt):
    while hbt.elapse(60 * 1e9) == 0:
        print('-------------------------------------------------------------------------------')
        print('current_timestamp:', hbt.current_timestamp)

        # Gets the last trades occurring in the market, not the trades of our orders.
        last_trades = hbt.last_trades(0)

        num = 0
        for last_trade in last_trades:
            if num > 10:
                print('...')
                break
            print(
                'exch_timestamp:',
                last_trade.exch_ts,
                'buy' if (last_trade.ev & BUY_EVENT) == BUY_EVENT else 'sell',
                last_trade.qty,
                '@',
                last_trade.px
            )
            num += 1

        # To prevent accumulating all last trades, which may cause a slowdown,
        # clear_last_trades needs to be called.
        # After this, accessing `last_trades` will cause a crash.
        hbt.clear_last_trades(0)
    return True

hbt = ROIVectorMarketDepthBacktest([asset])
print_last_trades(hbt)
_ = hbt.close()

-------------------------------------------------------------------------------
current_timestamp: 1717200060131000000
exch_timestamp: 1717200000131000000 sell 0.004 @ 67577.9
exch_timestamp: 1717200000136000000 sell 0.004 @ 67577.9
exch_timestamp: 1717200000140000000 sell 0.002 @ 67577.9
exch_timestamp: 1717200000140000000 sell 0.015 @ 67577.9
exch_timestamp: 1717200000219000000 sell 0.004 @ 67577.9
exch_timestamp: 1717200000254000000 sell 0.002 @ 67577.9
exch_timestamp: 1717200000308000000 buy 0.005 @ 67578.0
exch_timestamp: 1717200000308000000 buy 0.001 @ 67578.0
exch_timestamp: 1717200000377000000 sell 0.032 @ 67577.9
exch_timestamp: 1717200000383000000 sell 0.002 @ 67577.9
exch_timestamp: 1717200000384000000 sell 0.002 @ 67577.9
...
-------------------------------------------------------------------------------
current_timestamp: 1717200120131000000
exch_timestamp: 1717200061178000000 buy 0.06 @ 67608.7
exch_timestamp: 1717200061179000000 sell 0.116 @ 67608.6
exch_timestamp: 17172

In [7]:
@njit
def print_3depth(hbt):
    while hbt.elapse(60_000_000_000) == 0:
        print('current_timestamp:', hbt.current_timestamp)

        # Gets the market depth for the first asset, in the same order as when you created the backtest.
        depth = hbt.depth(0)

        # a key of bid_depth or ask_depth is price in ticks.
        # (integer) price_tick = rice / tick_size
        i = 0
        for price_tick in range(depth.best_ask_tick, depth.best_ask_tick + 100):
            qty = depth.ask_qty_at_tick(price_tick)
            if qty > 0:
                print(
                    'ask: ',
                    qty,
                    '@',
                    np.round(price_tick * depth.tick_size, 5)
                )

                i += 1
                if i == 3:
                    break
        i = 0
        for price_tick in range(depth.best_bid_tick, max(depth.best_bid_tick - 100, 0), -1):
            qty = depth.bid_qty_at_tick(price_tick)
            if qty > 0:
                print(
                    'bid: ',
                    qty,
                    '@',
                    np.round(price_tick * depth.tick_size, 5)
                )

                i += 1
                if i == 3:
                    break
    return True

hbt = ROIVectorMarketDepthBacktest([asset])
print_3depth(hbt)
_ = hbt.close()

current_timestamp: 1717200060131000000
ask:  8.054 @ 67608.7
bid:  1.346 @ 67608.6
current_timestamp: 1717200120131000000
ask:  11.524 @ 67597.1
bid:  1.543 @ 67597.0
current_timestamp: 1717200180131000000
ask:  5.661 @ 67619.6
bid:  0.279 @ 67619.5
current_timestamp: 1717200240131000000
ask:  1.574 @ 67641.6
bid:  8.642 @ 67641.5
current_timestamp: 1717200300131000000
ask:  0.907 @ 67680.4
bid:  15.946 @ 67680.3
current_timestamp: 1717200360131000000
ask:  0.39 @ 67675.1
bid:  8.783 @ 67675.0
current_timestamp: 1717200420131000000
ask:  2.723 @ 67675.3
bid:  4.568 @ 67675.2
current_timestamp: 1717200480131000000
ask:  14.337 @ 67697.1
bid:  3.361 @ 67697.0
current_timestamp: 1717200540131000000
ask:  13.375 @ 67678.2
bid:  2.424 @ 67678.1
current_timestamp: 1717200600131000000
ask:  11.086 @ 67641.7
bid:  1.919 @ 67641.6
current_timestamp: 1717200660131000000
ask:  11.676 @ 67655.0
bid:  7.847 @ 67654.9
current_timestamp: 1717200720131000000
ask:  3.511 @ 67644.4
bid:  3.738 @ 67644.3

In [8]:
@njit
def rolling_vwap(hbt, out):
    buy_amount_bin = np.zeros(100_000, np.float64)
    buy_qty_bin = np.zeros(100_000, np.float64)
    sell_amount_bin = np.zeros(100_000, np.float64)
    sell_qty_bin = np.zeros(100_000, np.float64)

    idx = 0
    last_trade_price = np.nan

    while hbt.elapse(10 * 1e9) == 0:
        last_trades = hbt.last_trades(0)

        for last_trade in last_trades:
            if (last_trade.ev & BUY_EVENT) == BUY_EVENT:
                buy_amount_bin[idx] += last_trade.px * last_trade.qty
                buy_qty_bin[idx] += last_trade.qty
            else:
                sell_amount_bin[idx] += last_trade.px * last_trade.qty
                sell_qty_bin[idx] += last_trade.qty

        hbt.clear_last_trades(0)
        idx += 1

        if idx >= 1:
            vwap10sec = np.divide(
                buy_amount_bin[idx - 1] + sell_amount_bin[idx - 1],
                buy_qty_bin[idx - 1] + sell_qty_bin[idx - 1]
            )
        else:
            vwap10sec = np.nan

        if idx >= 6:
            vwap1m = np.divide(
                np.sum(buy_amount_bin[idx - 6:idx]) + np.sum(sell_amount_bin[idx - 6:idx]),
                np.sum(buy_qty_bin[idx - 6:idx]) + np.sum(sell_qty_bin[idx - 6:idx])
            )
            buy_vwap1m = np.divide(np.sum(buy_amount_bin[idx - 6:idx]), np.sum(buy_qty_bin[idx - 6:idx]))
            sell_vwap1m = np.divide(np.sum(sell_amount_bin[idx - 6:idx]), np.sum(sell_qty_bin[idx - 6:idx]))
        else:
            vwap1m = np.nan
            buy_vwap1m = np.nan
            sell_vwap1m = np.nan

        out.append((hbt.current_timestamp, vwap10sec, vwap1m, buy_vwap1m, sell_vwap1m))
    return True

hbt = ROIVectorMarketDepthBacktest([asset])

tup_ty = Tuple((float64, float64, float64, float64, float64))
out = List.empty_list(tup_ty, allocated=100_000)

rolling_vwap(hbt, out)

_ = hbt.close()

df = pl.DataFrame(out).transpose()
df.columns = ['Local Timestamp', '10-sec VWAP', '1-min VWAP', '1-min Buy VWAP', '1-min Sell VWAP']
df = df.with_columns(
    pl.from_epoch('Local Timestamp', time_unit='ns')
)

df

Local Timestamp,10-sec VWAP,1-min VWAP,1-min Buy VWAP,1-min Sell VWAP
datetime[ns],f64,f64,f64,f64
2024-06-01 00:00:10.131000064,67582.80756,NaN,NaN,NaN
2024-06-01 00:00:20.131000064,67615.394129,NaN,NaN,NaN
2024-06-01 00:00:30.131000064,67609.689336,NaN,NaN,NaN
2024-06-01 00:00:40.131000064,67594.786063,NaN,NaN,NaN
2024-06-01 00:00:50.131000064,67606.6457,NaN,NaN,NaN
…,…,…,…,…
2024-06-01 23:59:10.131000064,67807.116229,67806.662051,67806.35261,67808.199673
2024-06-01 23:59:20.131000064,67807.19087,67807.911602,67807.806641,67808.394986
2024-06-01 23:59:30.131000064,67801.00283,67804.158476,67805.064797,67803.331082


In [9]:
df.hvplot(x='Local Timestamp')

:NdOverlay   [Variable]
   :Curve   [Local Timestamp]   (value)